In [0]:
%sql
-- Auto Loader → Bronze streaming table (SQL cloud_files).
-- Run after the Unity setup so Spark writes into the governed catalog and schema.
USE CATALOG main_nyctaxi;
-- The raw schema is the Bronze landing zone that holds immutable Delta tables for ingestion.
USE SCHEMA raw;

-- Spark Structured Streaming powers Auto Loader, so the streaming table stays in sync with new files.
CREATE OR REFRESH STREAMING TABLE taxi_raw
-- Delta handles ACID transactions and schema evolution for the Bronze layer.
TBLPROPERTIES (pipelines.autoOptimize.managed = true)
AS
SELECT * FROM cloud_files(
  'dbfs:/mnt/nyctaxi/input',          -- change to your mount or S3/ADLS URI
  'csv',
  map(
    'cloudFiles.inferColumnTypes','true',
    'cloudFiles.schemaEvolutionMode','addNewColumns',
    'cloudFiles.useNotifications','true'           -- Auto Loader watches object storage for new files
  )
);
